In [1]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import libraries to build the network
!pip install tensorflow

In [3]:
import os
import numpy as np
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import plot_model


In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# for processing the data before feeding it into the model

def data_generator(input_folder, ground_truth_folder, batch_size, input_size=(320, 640)):
    input_files = sorted(os.listdir(input_folder))
    ground_truth_files = sorted(os.listdir(ground_truth_folder))

    while True:
        for i in range(0, len(input_files), batch_size):
            batch_input_files = input_files[i:i + batch_size]
            batch_ground_truth_files = ground_truth_files[i:i + batch_size]

            batch_inputs = []
            batch_ground_truths = []

            for input_file, gt_file in zip(batch_input_files, batch_ground_truth_files):
                input_image = load_img(os.path.join(input_folder, input_file), color_mode="grayscale", target_size=input_size)
                input_image = img_to_array(input_image) / 255.0
                batch_inputs.append(input_image)

                ground_truth_image = load_img(os.path.join(ground_truth_folder, gt_file), color_mode="grayscale", target_size=input_size)
                ground_truth_image = img_to_array(ground_truth_image) / 255.0
                batch_ground_truths.append(ground_truth_image)

            yield np.array(batch_inputs), np.array(batch_ground_truths)

In [ ]:
# network implmentation

def attention_block(x, g, inter_channel):
    """
    Attention block for the U-Net.

    Args:
        x: Input tensor from the encoder.
        g: Input tensor from the decoder.
        inter_channel: Number of intermediate channels.
    Returns:
        Output tensor after applying attention.
    """
    theta_x = layers.Conv2D(inter_channel, (1, 1), padding="same")(x)
    phi_g = layers.Conv2D(inter_channel, (1, 1), padding="same")(g)
    f = layers.Activation('relu')(layers.Add()([theta_x, phi_g]))
    psi = layers.Conv2D(1, (1, 1), padding="same")(f)
    psi = layers.Activation('sigmoid')(psi)
    return layers.Multiply()([x, psi])

def attention_unet(input_shape=(320, 640, 1)):
    inputs = layers.Input(input_shape)

    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)

    u1 = layers.UpSampling2D((2, 2))(c4)
    att1 = attention_block(c3, u1, 128)
    u1 = layers.Concatenate()([u1, att1])
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c5)

    u2 = layers.UpSampling2D((2, 2))(c5)
    att2 = attention_block(c2, u2, 64)
    u2 = layers.Concatenate()([u2, att2])
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c6)

    u3 = layers.UpSampling2D((2, 2))(c6)
    att3 = attention_block(c1, u3, 32)
    u3 = layers.Concatenate()([u3, att3])
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u3)
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    return Model(inputs, outputs)

attention_model = attention_unet()


In [ ]:
attention_model.compile(optimizer=Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])

In [ ]:
# Paths
train_input_folder = os.path.join("/content/drive/MyDrive/Training", "input")
train_gt_folder = os.path.join("/content/drive/MyDrive/Training", "ground_truth")
val_input_folder = os.path.join("/content/drive/MyDrive/Validation", "input")
val_gt_folder = os.path.join("/content/drive/MyDrive/Validation", "ground_truth")

batch_size = 16
epochs = 1

train_gen = data_generator(train_input_folder, train_gt_folder, batch_size)
val_gen = data_generator(val_input_folder, val_gt_folder, batch_size)

train_steps = len(os.listdir(train_input_folder)) // batch_size
val_steps = len(os.listdir(val_input_folder)) // batch_size


In [ ]:
callbacks = [
    ModelCheckpoint("best_attention_model.keras", save_best_only=True, monitor="val_loss", mode="min"),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]


In [ ]:
#checking if the GPU is available
import tensorflow as tf
print("GPUs available:", len(tf.config.list_physical_devices('GPU')))

GPUs available: 1


In [ ]:
!nvidia-smi

Sun Jan  5 01:01:04 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
history = attention_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

435/435 ━━━━━━━━━━━━━━━━━━━━ 2675s 6s/step - accuracy: 0.0032 - loss: 0.6020 - val_accuracy: 0.0096 - val_loss: 0.5685


---------------------------------------

**ABOVE CODE WAS FOR T4/ BELOW IS L4**

---------------------------------------

In [ ]:
# Paths
train_input_folder = os.path.join("/content/drive/MyDrive/Training", "input")
train_gt_folder = os.path.join("/content/drive/MyDrive/Training", "ground_truth")
val_input_folder = os.path.join("/content/drive/MyDrive/Validation", "input")
val_gt_folder = os.path.join("/content/drive/MyDrive/Validation", "ground_truth")

batch_size = 24
epochs = 1

train_gen = data_generator(train_input_folder, train_gt_folder, batch_size)
val_gen = data_generator(val_input_folder, val_gt_folder, batch_size)

train_steps = len(os.listdir(train_input_folder)) // batch_size
val_steps = len(os.listdir(val_input_folder)) // batch_size

In [ ]:
print(train_steps)
print(val_steps)

290
68


In [ ]:
callbacks = [
    ModelCheckpoint("best_attention_model.keras", save_best_only=True, monitor="val_loss", mode="min"),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

class SaveEveryNEpochs(ModelCheckpoint):
    def __init__(self, save_freq, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            super().on_epoch_end(epoch, logs)

callbacks = [
    SaveEveryNEpochs(
        save_freq=52,  # Save every 5 epochs
        filepath="best_attention_model.keras",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    ),
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]

In [ ]:
history = attention_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

290/290 ━━━━━━━━━━━━━━━━━━━━ 1063s 3s/step - accuracy: 0.0047 - loss: 0.6164 - val_accuracy: 0.0097 - val_loss: 0.5750


In [ ]:
# got this error when using batch size 32,64

In [ ]:
---------------------------------------------------------------------------
ResourceExhaustedError                    Traceback (most recent call last)
<ipython-input-12-ae02d2b40640> in <cell line: 1>()
----> 1 history = attention_model.fit(
      2     train_gen,
      3     steps_per_epoch=train_steps,
      4     validation_data=val_gen,
      5     validation_steps=val_steps,

1 frames
/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/execute.py in quick_execute(op_name, num_outputs, inputs, attrs, ctx, name)
     51   try:
     52     ctx.ensure_initialized()
---> 53     tensors = pywrap_tfe.TFE_Py_Execute(ctx._handle, device_name, op_name,
     54                                         inputs, attrs, num_outputs)
     55   except core._NotOkStatusException as e:

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 738, in _run_callback

  File "/usr/local/lib/python3.10/dist-packages/tornado/gen.py", line 825, in inner

  File "/usr/local/lib/python3.10/dist-packages/tornado/gen.py", line 786, in run

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py", line 361, in process_one

  File "/usr/local/lib/python3.10/dist-packages/tornado/gen.py", line 234, in wrapper

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py", line 261, in dispatch_shell

  File "/usr/local/lib/python3.10/dist-packages/tornado/gen.py", line 234, in wrapper

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py", line 539, in execute_request

  File "/usr/local/lib/python3.10/dist-packages/tornado/gen.py", line 234, in wrapper

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py", line 302, in do_execute

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/zmqshell.py", line 539, in run_cell

  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2975, in run_cell

  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3030, in _run_cell

  File "/usr/local/lib/python3.10/dist-packages/IPython/core/async_helpers.py", line 78, in _pseudo_sync_runner

  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3257, in run_cell_async

  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3473, in run_ast_nodes

  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code

  File "<ipython-input-12-ae02d2b40640>", line 1, in <cell line: 1>

  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler

  File "/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py", line 320, in fit

  File "/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py", line 121, in one_step_on_iterator

Out of memory while trying to allocate 32681838224 bytes.
	 [[{{node StatefulPartitionedCall}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_one_step_on_iterator_8924]


--------------------------------

**Trying A100 GPU**

In [ ]:
# Set mixed precision policy (if you are using mixed precision)
import tensorflow as tf
from tensorflow.keras.models import load_model

from tensorflow.keras.mixed_precision import Policy, set_global_policy


model_path = "/content/drive/MyDrive/best_attention_model.keras"  # Update with your saved model path
attention_model = load_model(model_path, compile=True)

attention_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# Paths
train_input_folder = os.path.join("/content/drive/MyDrive/Training", "input")
train_gt_folder = os.path.join("/content/drive/MyDrive/Training", "ground_truth")
val_input_folder = os.path.join("/content/drive/MyDrive/Validation", "input")
val_gt_folder = os.path.join("/content/drive/MyDrive/Validation", "ground_truth")

batch_size = 32
epochs = 1

train_gen = data_generator(train_input_folder, train_gt_folder, batch_size)
val_gen = data_generator(val_input_folder, val_gt_folder, batch_size)

train_steps = len(os.listdir(train_input_folder)) // batch_size
val_steps = len(os.listdir(val_input_folder)) // batch_size

In [ ]:
print(train_steps)
print(val_steps)

217
51


In [ ]:
callbacks = [
    ModelCheckpoint("best_attention_model.keras", save_best_only=True, monitor="val_loss", mode="min"),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

In [ ]:
history = attention_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs
)

 28/217 ━━━━━━━━━━━━━━━━━━━━ 44:31 14s/step - accuracy: 4.5853e-04 - loss: 0.5662

KeyboardInterrupt: 

---------------------------------------------------------------------------

**FINAL TRAINING RUN**

---------------------------------------------------------------------------

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
# Load the saved model
model_path = "/content/drive/MyDrive/best_attention_model.keras"  # Update with your saved model path
attention_model = load_model(model_path, compile=True)

In [ ]:
# Confirming the previous loss and optimizer configuration was loaded
print("Optimizer:", attention_model.optimizer.get_config())
print("Loss function:", attention_model.loss)

Optimizer: {'name': 'adam', 'learning_rate': 9.999999747378752e-05, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
Loss function: binary_crossentropy


In [ ]:
########################## FINAL DATA LOADER #############################
train_input_folder = os.path.join("/content/drive/MyDrive/Training", "input")
train_gt_folder = os.path.join("/content/drive/MyDrive/Training", "ground_truth")
val_input_folder = os.path.join("/content/drive/MyDrive/Validation", "input")
val_gt_folder = os.path.join("/content/drive/MyDrive/Validation", "ground_truth")

batch_size = 24

train_gen = data_generator(train_input_folder, train_gt_folder, batch_size)
val_gen = data_generator(val_input_folder, val_gt_folder, batch_size)

train_steps = len(os.listdir(train_input_folder)) // batch_size
val_steps = len(os.listdir(val_input_folder)) // batch_size

In [ ]:
print(train_steps)
print(val_steps)

290
68


In [ ]:
callbacks = [
    ModelCheckpoint("best_attention_model.keras", save_best_only=True, monitor="val_loss", mode="min"),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

class SaveEveryNEpochs(ModelCheckpoint):
    def __init__(self, save_freq, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            super().on_epoch_end(epoch, logs)

callbacks = [
    SaveEveryNEpochs(
        save_freq=5,  # Save every 5 epochs
        filepath="best_attention_model.keras",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    ),
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]

In [ ]:
history = attention_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/52
  5/290 ━━━━━━━━━━━━━━━━━━━━ 3:58 837ms/step - accuracy: 4.3703e-04 - loss: 0.5605

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:199: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=self._current_epoch, batch=batch, logs=logs)


290/290 ━━━━━━━━━━━━━━━━━━━━ 1532s 5s/step - accuracy: 0.0047 - loss: 0.5609 - val_accuracy: 0.0097 - val_loss: 0.5712
Epoch 2/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 303s 984ms/step - accuracy: 0.0047 - loss: 0.5570 - val_accuracy: 0.0097 - val_loss: 0.5608
Epoch 3/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 262s 903ms/step - accuracy: 0.0046 - loss: 0.5532 - val_accuracy: 0.0097 - val_loss: 0.5589
Epoch 4/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 260s 898ms/step - accuracy: 0.0046 - loss: 0.5511 - val_accuracy: 0.0097 - val_loss: 0.5574
Epoch 5/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 260s 897ms/step - accuracy: 0.0046 - loss: 0.5496 - val_accuracy: 0.0097 - val_loss: 0.5561
Epoch 6/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 260s 897ms/step - accuracy: 0.0045 - loss: 0.5489 - val_accuracy: 0.0097 - val_loss: 0.5560
Epoch 7/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 260s 897ms/step - accuracy: 0.0045 - loss: 0.5486 - val_accuracy: 0.0097 - val_loss: 0.5550
Epoch 8/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 260s 898ms/step - accuracy: 0.0044 - loss: 0.5476

In [ ]:
!nvidia-smi

Sun Jan  5 13:54:55 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   70C    P0              32W /  72W |  21207MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

------------------------------------------------------------------------------

**Training Run After Changes**

-----------------------------------------------------------------------------

In [ ]:
callbacks = [
    ModelCheckpoint("best_attention_model.keras", save_best_only=True, monitor="val_loss", mode="min"),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

class SaveEveryNEpochs(ModelCheckpoint):
    def __init__(self, save_freq, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            super().on_epoch_end(epoch, logs)

callbacks = [
    SaveEveryNEpochs(
        save_freq=5,  # Save every 5 epochs
        filepath="/content/drive/MyDrive/best_new_attention_model.keras",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    ),
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)
callbacks.append(reduce_lr)

In [ ]:
# tweaking the attention block and changing model architecture

def attention_block(x, g, inter_channel):
    theta_x = layers.Conv2D(inter_channel, (1, 1), padding="same")(x)
    phi_g = layers.Conv2D(inter_channel, (1, 1), padding="same")(g)
    f = layers.Activation('relu')(layers.Add()([theta_x, phi_g]))
    f = layers.Conv2D(inter_channel, (3, 3), padding="same", activation="relu")(f)  # Extra layer
    psi = layers.Conv2D(1, (1, 1), padding="same")(f)
    psi = layers.Activation('sigmoid')(psi)
    return layers.Multiply()([x, psi])


def attention_unet(input_shape=(320, 640, 1)):
    inputs = layers.Input(input_shape)

    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)  # New layer

    u1 = layers.UpSampling2D((2, 2))(c4)
    att1 = attention_block(c3, u1, 128)
    u1 = layers.Concatenate()([u1, att1])
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c5)

    u2 = layers.UpSampling2D((2, 2))(c5)
    att2 = attention_block(c2, u2, 64)
    u2 = layers.Concatenate()([u2, att2])
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c6)

    u3 = layers.UpSampling2D((2, 2))(c6)
    att3 = attention_block(c1, u3, 32)
    u3 = layers.Concatenate()([u3, att3])
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u3)
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    return Model(inputs, outputs)

attention_model = attention_unet()


In [ ]:
attention_model.compile(optimizer=Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["accuracy"])

In [ ]:
history = attention_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/52
  5/290 ━━━━━━━━━━━━━━━━━━━━ 4:44 998ms/step - accuracy: 4.9237e-04 - loss: 0.0766

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:199: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=self._current_epoch, batch=batch, logs=logs)


290/290 ━━━━━━━━━━━━━━━━━━━━ 328s 1s/step - accuracy: 0.0041 - loss: 0.0405 - val_accuracy: 0.0097 - val_loss: 0.0178 - learning_rate: 1.0000e-04
Epoch 2/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.0014 - loss: 0.0174 - val_accuracy: 0.0097 - val_loss: 0.0176 - learning_rate: 1.0000e-04
Epoch 3/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.0037 - loss: 0.0155 - val_accuracy: 0.0097 - val_loss: 0.0173 - learning_rate: 1.0000e-04
Epoch 4/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.0040 - loss: 0.0140 - val_accuracy: 0.0097 - val_loss: 0.0170 - learning_rate: 1.0000e-04
Epoch 5/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.0039 - loss: 0.0131 - val_accuracy: 0.0097 - val_loss: 0.0162 - learning_rate: 1.0000e-04
Epoch 6/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.0039 - loss: 0.0126 - val_accuracy: 0.0097 - val_loss: 0.0149 - learning_rate: 1.0000e-04
Epoch 7/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.00

KeyboardInterrupt: 

In [ ]:
attention_model.save('/content/drive/MyDrive/model_with_mse.keras')

In [ ]:
#changing the loss function to ssim_loss and continuing training

In [ ]:
attention_model = load_model("/content/drive/MyDrive/model_with_mse.keras")

In [ ]:
import tensorflow as tf

def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))

In [ ]:
attention_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=ssim_loss,
    metrics=[tf.image.psnr]  # Add PSNR for monitoring quality
)

In [ ]:
history = attention_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/52
  5/290 ━━━━━━━━━━━━━━━━━━━━ 4:49 1s/step - accuracy: 3.7150e-04 - loss: 0.0075

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:199: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=self._current_epoch, batch=batch, logs=logs)


290/290 ━━━━━━━━━━━━━━━━━━━━ 334s 1s/step - accuracy: 0.0048 - loss: 0.0100 - val_accuracy: 0.0097 - val_loss: 0.0123 - learning_rate: 1.0000e-04
Epoch 2/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.0047 - loss: 0.0101 - val_accuracy: 0.0097 - val_loss: 0.0126 - learning_rate: 1.0000e-04
Epoch 3/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.0047 - loss: 0.0101 - val_accuracy: 0.0097 - val_loss: 0.0124 - learning_rate: 1.0000e-04
Epoch 4/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0047 - loss: 0.0100
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
290/290 ━━━━━━━━━━━━━━━━━━━━ 310s 1s/step - accuracy: 0.0047 - loss: 0.0100 - val_accuracy: 0.0097 - val_loss: 0.0123 - learning_rate: 1.0000e-04
Epoch 5/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 309s 1s/step - accuracy: 0.0046 - loss: 0.0099 - val_accuracy: 0.0097 - val_loss: 0.0127 - learning_rate: 5.0000e-05
Epoch 6/52
290/290 ━━━━━━━━━━━━━━━━━━━━ 309s 1s/step - accuracy: 0.0046 - lo

KeyboardInterrupt: 

In [ ]:
attention_model.save('/content/drive/MyDrive/model_with_ssim.keras')

-----------------------------------------------------------------------------

**TRAINED FOR (14.2+13.2) EPOCHS, TRAINING FOR MORE**

----------------------------------------------------------------------------

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
    # Encoder
def attention_block(x, g, inter_channel):
    theta_x = layers.Conv2D(inter_channel, (1, 1), padding="same")(x)
    phi_g = layers.Conv2D(inter_channel, (1, 1), padding="same")(g)
    f = layers.Activation('relu')(layers.Add()([theta_x, phi_g]))
    f = layers.Conv2D(inter_channel, (3, 3), padding="same", activation="relu")(f)  # Extra layer
    psi = layers.Conv2D(1, (1, 1), padding="same")(f)
    psi = layers.Activation('sigmoid')(psi)
    return layers.Multiply()([x, psi])


def attention_unet(input_shape=(320, 640, 1)):
    inputs = layers.Input(input_shape)

    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)  # New layer

    u1 = layers.UpSampling2D((2, 2))(c4)
    att1 = attention_block(c3, u1, 128)
    u1 = layers.Concatenate()([u1, att1])
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c5)

    u2 = layers.UpSampling2D((2, 2))(c5)
    att2 = attention_block(c2, u2, 64)
    u2 = layers.Concatenate()([u2, att2])
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c6)

    u3 = layers.UpSampling2D((2, 2))(c6)
    att3 = attention_block(c1, u3, 32)
    u3 = layers.Concatenate()([u3, att3])
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u3)
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    return Model(inputs, outputs)

In [ ]:
extra_20_epoch_model = attention_unet()

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op FloorMod in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:loca

In [ ]:
extra_20_epoch_model = load_model("/content/drive/MyDrive/model_with_mse.keras")

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op FloorMod in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:loca

In [ ]:
import tensorflow as tf

def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))

In [ ]:
attention_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=ssim_loss,
    metrics=[tf.image.psnr]  # Add PSNR for monitoring quality
)

In [ ]:
callbacks = [
    ModelCheckpoint("best_attention_model.keras", save_best_only=True, monitor="val_loss", mode="min"),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

class SaveEveryNEpochs(ModelCheckpoint):
    def __init__(self, save_freq, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            super().on_epoch_end(epoch, logs)

callbacks = [
    SaveEveryNEpochs(
        save_freq=3,  # Save every 5 epochs
        filepath="/content/drive/MyDrive/best_new_attention_model.keras",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    ),
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]


from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)
callbacks.append(reduce_lr)

In [ ]:
epochs = 20

In [ ]:
# STOPPED THIS TRAINING RUN CUZ MAYBE NOT COMPILING THE MODEL MADE THE EPOCHS GO
history = extra_20_epoch_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)


Epoch 1/20
  5/290 ━━━━━━━━━━━━━━━━━━━━ 2:23:30 30s/step - accuracy: 4.4194e-04 - loss: 0.0084

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:199: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=self._current_epoch, batch=batch, logs=logs)


 27/290 ━━━━━━━━━━━━━━━━━━━━ 2:39:53 36s/step - accuracy: 4.5009e-04 - loss: 0.0086

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)

Num GPUs Available:  1


In [ ]:
extra_20_epoch_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=ssim_loss,
    metrics=[tf.image.psnr]  # Add PSNR for monitoring quality
)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
history = extra_20_epoch_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Epoch 1/20
Executing op AnonymousIteratorV3 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
E

TypeError: Missing required positional argument

In [ ]:
new_extra_20_epoch_model = load_model("/content/drive/MyDrive/model_with_mse.keras")

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op FloorMod in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:loca

In [ ]:
history = new_extra_20_epoch_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Epoch 1/20
Executing op AnonymousIteratorV3 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/device:

In [ ]:
# Suppress TensorFlow logging
import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress info and warning messages
tf.get_logger().setLevel('ERROR')         # Suppress TensorFlow execution logs


In [ ]:
final_model = load_model("/content/drive/MyDrive/model_with_ssim.keras")

In [ ]:
epochs = 25

In [ ]:
history = final_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/25
  3/290 ━━━━━━━━━━━━━━━━━━━━ 2:14:25 28s/step - accuracy: 4.5341e-04 - loss: 0.0078

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:199: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=self._current_epoch, batch=batch, logs=logs)


290/290 ━━━━━━━━━━━━━━━━━━━━ 499s 1s/step - accuracy: 0.0047 - loss: 0.0091 - val_accuracy: 0.0097 - val_loss: 0.0118 - learning_rate: 1.2500e-05
Epoch 2/25
290/290 ━━━━━━━━━━━━━━━━━━━━ 352s 1s/step - accuracy: 0.0047 - loss: 0.0094 - val_accuracy: 0.0097 - val_loss: 0.0117 - learning_rate: 1.2500e-05
Epoch 3/25
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0046 - loss: 0.0095 - val_accuracy: 0.0097 - val_loss: 0.0117 - learning_rate: 1.2500e-05
Epoch 4/25
290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0046 - loss: 0.0094
Epoch 4: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0046 - loss: 0.0095 - val_accuracy: 0.0097 - val_loss: 0.0117 - learning_rate: 1.2500e-05
Epoch 5/25
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0046 - loss: 0.0095 - val_accuracy: 0.0097 - val_loss: 0.0115 - learning_rate: 6.2500e-06
Epoch 6/25
 13/290 ━━━━━━━━━━━━━━━━━━━━ 4:19 935ms/step - accuracy: 4.3430e-0

KeyboardInterrupt: 

In [ ]:
final_model.save('/content/drive/MyDrive/final_model.keras')

In [ ]:
final_model = load_model("/content/drive/MyDrive/final_model.keras")

In [ ]:
epochs = 20

In [ ]:
history = final_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/20
  1/290 ━━━━━━━━━━━━━━━━━━━━ 42:57 9s/step - accuracy: 4.5797e-04 - loss: 0.0091

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:199: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=self._current_epoch, batch=batch, logs=logs)


290/290 ━━━━━━━━━━━━━━━━━━━━ 325s 1s/step - accuracy: 0.0052 - loss: 0.0094 - val_accuracy: 0.0097 - val_loss: 0.0110 - learning_rate: 6.2500e-06
Epoch 2/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0051 - loss: 0.0093 - val_accuracy: 0.0097 - val_loss: 0.0108 - learning_rate: 6.2500e-06
Epoch 3/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0051 - loss: 0.0093 - val_accuracy: 0.0097 - val_loss: 0.0108 - learning_rate: 6.2500e-06
Epoch 4/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0050 - loss: 0.0093 - val_accuracy: 0.0097 - val_loss: 0.0108 - learning_rate: 6.2500e-06
Epoch 5/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0050 - loss: 0.0093
Epoch 5: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-06.
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0050 - loss: 0.0093 - val_accuracy: 0.0097 - val_loss: 0.0109 - learning_rate: 6.2500e-06
Epoch 6/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0049 - los

In [ ]:
final_model.save('/content/drive/MyDrive/final_model_1.keras')

In [ ]:
history = final_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 312s 1s/step - accuracy: 0.0049 - loss: 0.0094 - val_accuracy: 0.0097 - val_loss: 0.0113 - learning_rate: 1.5625e-06
Epoch 2/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0049 - loss: 0.0094 - val_accuracy: 0.0097 - val_loss: 0.0113 - learning_rate: 1.5625e-06
Epoch 3/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0048 - loss: 0.0094 - val_accuracy: 0.0097 - val_loss: 0.0113 - learning_rate: 1.5625e-06
Epoch 4/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0048 - loss: 0.0093
Epoch 4: ReduceLROnPlateau reducing learning rate to 1e-06.
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0048 - loss: 0.0093 - val_accuracy: 0.0097 - val_loss: 0.0113 - learning_rate: 1.5625e-06
Epoch 5/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0047 - loss: 0.0094 - val_accuracy: 0.0097 - val_loss: 0.0112 - learning_rate: 1.0000e-06


In [ ]:
final_model.save('/content/drive/MyDrive/final_model_2.keras')

In [ ]:
history = final_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 312s 1s/step - accuracy: 0.0048 - loss: 0.0095 - val_accuracy: 0.0097 - val_loss: 0.0112 - learning_rate: 1.0000e-06
Epoch 2/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0047 - loss: 0.0094 - val_accuracy: 0.0097 - val_loss: 0.0112 - learning_rate: 1.0000e-06
Epoch 3/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 312s 1s/step - accuracy: 0.0047 - loss: 0.0094 - val_accuracy: 0.0097 - val_loss: 0.0112 - learning_rate: 1.0000e-06
Epoch 4/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0047 - loss: 0.0094 - val_accuracy: 0.0097 - val_loss: 0.0111 - learning_rate: 1.0000e-06
Epoch 5/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.0046 - loss: 0.0095 - val_accuracy: 0.0097 - val_loss: 0.0112 - learning_rate: 1.0000e-06


In [ ]:
final_model.save('/content/drive/MyDrive/final_model_3.keras')

In [ ]:
callbacks = [
    ModelCheckpoint("best_attention_model.keras", save_best_only=True, monitor="val_loss", mode="min"),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

class SaveEveryNEpochs(ModelCheckpoint):
    def __init__(self, save_freq, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            super().on_epoch_end(epoch, logs)

callbacks = [
    SaveEveryNEpochs(
        save_freq=3,  # Save every 5 epochs
        filepath="/content/drive/MyDrive/best_new_attention_model.keras",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    ),
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]


from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-6,
    verbose=1
)
callbacks.append(reduce_lr)

In [ ]:
history = final_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

In [ ]:
#

In [ ]:
# try re-compiling and starting if loss and validation loss isnt increasing.

In [ ]:
recompiled_model = load_model("/content/drive/MyDrive/model_with_ssim.keras")

In [ ]:
# didnt use the one below ---> for some reason the one below was throwing error during
recompiled_model.compile(optimizer=Adam(learning_rate=1e-5), loss=ssim_loss)




In [ ]:
history = recompiled_model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/30
 56/290 ━━━━━━━━━━━━━━━━━━━━ 4:00 1s/step - loss: 0.4845

KeyboardInterrupt: 

----------------------------------------------------------------------------

----------------------------------------------------------------------------

----------------------------------------------------------------------------

----------------------------------------------------------------------------





**CODE BELOW IS OPTIMISED FOR GPU L4/A100 (AINT WORKING THO)**

------------------------------------------------------------------

In [ ]:
class SaveEveryNEpochs(ModelCheckpoint):
    def __init__(self, save_freq, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            super().on_epoch_end(epoch, logs)

callbacks = [
    SaveEveryNEpochs(
        save_freq=5,  # Save every 5 epochs
        filepath="best_attention_model.keras",
        save_best_only=True,
        monitor="val_loss",
        mode="min"
    ),
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]

In [ ]:
# Enable mixed precision
from tensorflow.keras.mixed_precision import set_global_policy, Policy

policy = Policy('mixed_float16')  # Use mixed precision policy
set_global_policy(policy)

print("Mixed Precision Policy:", policy)

Mixed Precision Policy: <DTypePolicy "mixed_float16">


In [ ]:
# Paths
train_input_paths = os.path.join("/content/drive/MyDrive/Training", "input")
train_target_paths = os.path.join("/content/drive/MyDrive/Training", "ground_truth")
val_input_paths = os.path.join("/content/drive/MyDrive/Validation", "input")
val_target_paths = os.path.join("/content/drive/MyDrive/Validation", "ground_truth")

train_input_folder = os.path.join("/content/drive/MyDrive/Training", "input")
val_input_folder = os.path.join("/content/drive/MyDrive/Validation", "input")
BATCH_SIZE = 24
epochs = 6


train_steps = len(os.listdir(train_input_folder)) // BATCH_SIZE
val_steps = len(os.listdir(val_input_folder)) // BATCH_SIZE


In [ ]:
print(train_steps)
print(val_steps)

290
68


In [ ]:
# Create Dataset
import tensorflow as tf

def create_dataset(input_paths, target_paths, batch_size):
    def load_data(input_path, target_path):
        input_img = tf.image.decode_png(tf.io.read_file(input_path), channels=1)
        target_img = tf.image.decode_png(tf.io.read_file(target_path), channels=1)
        input_img = tf.image.resize(input_img, (320, 640)) / 255.0
        target_img = tf.image.resize(target_img, (320, 640)) / 255.0
        return input_img, target_img

    dataset = tf.data.Dataset.from_tensor_slices((input_paths, target_paths))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache()
    dataset = dataset.shuffle(len(input_paths))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = create_dataset(train_input_paths, train_target_paths, batch_size=BATCH_SIZE)
val_dataset = create_dataset(val_input_paths, val_target_paths, batch_size=BATCH_SIZE)


ValueError: Unbatching a tensor is only supported for rank >= 1

In [ ]:
# Example file paths
import tensorflow as tf
import os

train_input_paths = ["/content/drive/MyDrive/Training/input/MRIT_450_slice_013.png", "/content/drive/MyDrive/Training/input/MRIT_450_slice_012.png"]  # Replace with actual file paths
train_target_paths = ["/content/drive/MyDrive/Training/ground_truth/MRIT_450_slice_013.png", "/content/drive/MyDrive/Training/ground_truth/MRIT_450_slice_012.png"]  # Replace with actual file paths
BATCH_SIZE = 16

def create_dataset(input_paths, target_paths, batch_size):
    def load_data(input_path, target_path):
        input_img = tf.image.decode_png(tf.io.read_file(input_path), channels=1)
        target_img = tf.image.decode_png(tf.io.read_file(target_path), channels=1)
        input_img = tf.image.resize(input_img, (320, 640)) / 255.0
        target_img = tf.image.resize(target_img, (320, 640)) / 255.0
        return input_img, target_img

    assert len(input_paths) == len(target_paths), "Input and target paths must have the same length"

    dataset = tf.data.Dataset.from_tensor_slices((input_paths, target_paths))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache()
    dataset = dataset.shuffle(len(input_paths))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

print("Train Input Paths:", train_input_paths[:5])
print("Train Target Paths:", train_target_paths[:5])

train_dataset = create_dataset(train_input_paths, train_target_paths, batch_size=BATCH_SIZE)


Train Input Paths: ['/content/drive/MyDrive/Training/input/MRIT_450_slice_013.png', '/content/drive/MyDrive/Training/input/MRIT_450_slice_012.png']
Train Target Paths: ['/content/drive/MyDrive/Training/ground_truth/MRIT_450_slice_013.png', '/content/drive/MyDrive/Training/ground_truth/MRIT_450_slice_012.png']


In [ ]:
# Example file paths
import tensorflow as tf
import os

BATCH_SIZE = 64
train_input_paths = sorted([f"/content/drive/MyDrive/Training/input/{f}" for f in os.listdir("/content/drive/MyDrive/Training/input")])
train_target_paths = sorted([f"/content/drive/MyDrive/Training/ground_truth/{f}" for f in os.listdir("/content/drive/MyDrive/Training/ground_truth")])
val_input_paths = sorted([f"/content/drive/MyDrive/Validation/input/{f}" for f in os.listdir("/content/drive/MyDrive/Validation/input")])
val_target_paths = sorted([f"/content/drive/MyDrive/Validation/ground_truth/{f}" for f in os.listdir("/content/drive/MyDrive/Validation/ground_truth")])

print(f"Number of input paths: {len(train_input_paths)}")
print(f"Number of target paths: {len(train_target_paths)}")
assert len(train_input_paths) == len(train_target_paths), "Input and target path lists are not of the same length!"

for path in train_input_paths + train_target_paths:
    if not tf.io.gfile.exists(path):
        print(f"File not found: {path}")

def create_dataset(input_paths, target_paths, batch_size):
    def load_data(input_path, target_path):
        input_img = tf.image.decode_png(tf.io.read_file(input_path), channels=1)
        target_img = tf.image.decode_png(tf.io.read_file(target_path), channels=1)
        input_img = tf.image.resize(input_img, (320, 640)) / 255.0
        target_img = tf.image.resize(target_img, (320, 640)) / 255.0
        return input_img, target_img

    dataset = tf.data.Dataset.from_tensor_slices((input_paths, target_paths))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache()
    dataset = dataset.shuffle(len(input_paths))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = create_dataset(train_input_paths, train_target_paths, batch_size=BATCH_SIZE)
val_dataset = create_dataset(train_input_paths, train_target_paths, batch_size=BATCH_SIZE)


Number of input paths: 6964
Number of target paths: 6964


In [ ]:
print(f"Number of input paths: {len(val_input_paths)}")
print(f"Number of target paths: {len(val_target_paths)}")

Number of input paths: 1652
Number of target paths: 1652


In [ ]:
#for input_img, target_img in train_dataset.take(1):  # Fetch one batch

In [ ]:
# Train using Dataset
history = attention_model.fit(
    train_dataset,
    epochs=6,
    validation_data=val_dataset,
    callbacks=callbacks
)

Epoch 1/6


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/pyt

In [ ]:
## to later reload the model

from tensorflow.keras.models import load_model

saved_model = load_model("best_attention_model.keras")

inputs = saved_model.input
outputs = saved_model.layers[-2].output

new_outputs = layers.Conv2D(1, (1, 1), activation='sigmoid', dtype='float32')(outputs)

new_model = Model(inputs=inputs, outputs=new_outputs)

new_model.save("best_attention_model_updated.keras")

ValueError: File not found: filepath=best_attention_model.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
# Create the Attention U-Net model
from tensorflow.keras.utils import plot_model

attention_model = attention_unet(input_shape=(512, 512, 1))

attention_model.summary()

plot_model(attention_model, to_file="attention_unet.png", show_shapes=True, dpi=100)


In [ ]:
# specify input to the layer

In [ ]:
# break it down into batchsize

In [ ]:
# reduced model for later

In [ ]:
    # Encoder
def attention_block(x, g, inter_channel):
    """
    Attention block for the U-Net.

    Args:
        x: Input tensor from the encoder.
        g: Input tensor from the decoder.
        inter_channel: Number of intermediate channels.
    Returns:
        Output tensor after applying attention.
    """
    theta_x = layers.Conv2D(inter_channel, (1, 1), padding="same")(x)
    phi_g = layers.Conv2D(inter_channel, (1, 1), padding="same")(g)
    f = layers.Activation('relu')(layers.Add()([theta_x, phi_g]))
    psi = layers.Conv2D(1, (1, 1), padding="same")(f)
    psi = layers.Activation('sigmoid')(psi)
    return layers.Multiply()([x, psi])


def attention_unet_reduced(input_shape=(320, 640, 1)):
    """
    Reduced Attention U-Net architecture with 2 encoder blocks, 1 bottleneck, and 2 decoder blocks.

    Args:
        input_shape: Shape of the input images (height, width, channels).
    Returns:
        Compiled Keras model.
    """
    inputs = layers.Input(input_shape)

    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)

    u1 = layers.UpSampling2D((2, 2))(c3)
    att1 = attention_block(c2, u1, 64)
    u1 = layers.Concatenate()([u1, att1])
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    u2 = layers.UpSampling2D((2, 2))(c4)
    att2 = attention_block(c1, u2, 32)
    u2 = layers.Concatenate()([u2, att2])
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c5)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c5)

    return Model(inputs, outputs)


attention_model_reduced = attention_unet_reduced()
attention_model_reduced.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 320, 640, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_48 (Conv2D)        │ (None, 320, 640, 32)   │            320 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_49 (Conv2D)        │ (None, 320, 640, 32)   │          9,248 │ conv2d_48[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_6           │ (None, 160, 320, 32)   │              0 │ conv2d_49[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_50 (Conv2D)        │ (None, 160, 320, 64)   │         18,496 │ max_pooling2d_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_51 (Conv2D)        │ (None, 160, 320, 64)   │         36,928 │ conv2d_50[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_7           │ (None, 80, 160, 64)    │              0 │ conv2d_51[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_52 (Conv2D)        │ (None, 80, 160, 128)   │         73,856 │ max_pooling2d_7[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_53 (Conv2D)        │ (None, 80, 160, 128)   │        147,584 │ conv2d_52[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_6           │ (None, 160, 320, 128)  │              0 │ conv2d_53[0][0]        │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_54 (Conv2D)        │ (None, 160, 320, 64)   │          4,160 │ conv2d_51[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_55 (Conv2D)        │ (None, 160, 320, 64)   │          8,256 │ up_sampling2d_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_6 (Add)               │ (None, 160, 320, 64)   │              0 │ conv2d_54[0][0],       │
│                           │                        │                │ conv2d_55[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_12             │ (None, 160, 320, 64)   │              0 │ add_6[0][0]            │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_56 (Conv2D)        │ (None, 160, 320, 1)    │             65 │ activation_12[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_13             │ (None, 160, 320, 1)    │              0 │ conv2d_56[0][0]        │
│ (Activation)              │                        │                │                        │
├──────────────────────

 Total params: 486,627 (1.86 MB)

 Trainable params: 486,627 (1.86 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 82.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found exis